## Detecting anomalies with Random Cut Forest
Random Cut Forest (RCF) is an unsupervised learning algorithm for anomaly detection
( https://proceedings.mlr.press/v48/guha16.pdf ). We're going to apply
it to a subset of the household electric power consumption dataset ( https://archive.
ics.uci.edu/ml/ ), available in the GitHub repository for this book. The data has been
aggregated hourly over a period of little less than a year (just under 8,000 values):

In [80]:
# reading data from URL
import pandas as pd
url = "https://github.com/PacktPublishing/Learn-Amazon-SageMaker/blob/master/sdkv2/ch4/item-demand-time.csv?raw=true"
# ?raw=true (puted this on the end of the link)
data = pd.read_csv(url)
display(data.head(3))

# saving file in current path
data.to_csv('item-demand-time.csv')

,2014-01-01 01:00:00,38.34991708126038,client_12
0,2014-01-01 02:00:00,33.582090,client_12
1,2014-01-01 03:00:00,34.411277,client_12
2,2014-01-01 04:00:00,39.800995,client_12


In [81]:
import pandas as pd

df = pd.read_csv('item-demand-time.csv', dtype = object, names=['timestamp','value','item'],skiprows=1)
df.head(3)

,timestamp,value,item
0,2014-01-01 02:00:00,33.5820895522388,client_12
1,2014-01-01 03:00:00,34.41127694859037,client_12
2,2014-01-01 04:00:00,39.800995024875625,client_12


In [82]:
df.item.unique()

array(['client_12', 'client_10', 'client_111'], dtype=object)

In [84]:
# giving me error for some unkonwn reason
#%matplotlib inline

#import matplotlib
#import matplotlib.pyplot as plt

#df.value=pd.to_numeric(df.value)
#df_plot = df.pivot(index='timestamp', columns='item', values='value')
#df_plot.plot(figsize=(40,10))

The plot is shown in the following diagram. We see three time series corresponding
to three different clients:

There are two issues with this dataset. First, it contains several time series: RCF
can only train a model on a single series. Second, RCF requires integer values.
Let's solve both problem with pandas : we only keep the "client_12" time
series, we multiply its values by 100, and cast them to the integer type:

In [85]:
df = df[df['item']=='client_12']
df = df.drop(['item', 'timestamp'], axis=1)

In [86]:
df.value = df.value.astype('float32')
df.value*=100
df.value = df.value.astype('int32')
print("\nThe following diagram shows the first lines of the transformed dataset:")
df.head(3)


The following diagram shows the first lines of the transformed dataset:


,value
0,3358
1,3441
2,3980


In [87]:
#df.plot(figsize=(40,10))

In [88]:
# saving it to upload it to s3 later
df.to_csv('electricity.csv', index=False, header=False)

In [91]:
################## Extra step for local user only

import boto3
region = boto3.Session().region_name

def resolve_sm_role():
    client = boto3.client('iam', region_name=region)
    response_roles = client.list_roles(
        PathPrefix='/',
        # Marker='string',
        MaxItems=999
    )
    for role in response_roles['Roles']:
        if role['RoleName'].startswith('AmazonSageMaker-ExecutionRole-'):
            #print('Resolved SageMaker IAM Role to: ' + str(role))
            return role['Arn']
    raise Exception('Could not resolve what should be the SageMaker role to be used')

role = resolve_sm_role()
print(role)

#################

arn:aws:iam::603012210694:role/service-role/AmazonSageMaker-ExecutionRole-20210304T123661


Then, we define the training channel. There are a couple of quirks that we haven't
met before. SageMaker generally doesn't have many of these, and reading the
documentation goes a long way in pinpointing them ( https://docs.aws.
amazon.com/sagemaker/latest/dg/randomcutforest.html ).
First, the content type must state that data is not labeled. The reason for this is that
RCF can accept an optional test channel where anomalies are labeled ( label_
size=1 ). Even though the training channel never has labels, we still need to tell
RCF. Second, the only distribution policy supported in RCF is ShardedByS3Key .
This policy splits the dataset across the different instances in the training cluster,
instead of sending them a full copy. We won't run distributed training here, but we
need to set that policy nonetheless

In [92]:
import boto3
import sagemaker

print(sagemaker.__version__)

sess = sagemaker.Session()
#role = sagemaker.get_execution_role()
bucket = sess.default_bucket()
prefix = 'electricity'

training_data_path = sess.upload_data(path='electricity.csv', key_prefix=prefix + '/input/training')
training_data_channel = sagemaker.TrainingInput(s3_data=training_data_path, 
                                           content_type='text/csv;label_size=0',
                                           distribution='ShardedByS3Key')
rcf_data = {'train': training_data_channel}

2.26.0


In [93]:
print(training_data_path)

s3://sagemaker-us-east-1-603012210694/electricity/input/training/electricity.csv


In [94]:
import boto3
from sagemaker.estimator import Estimator
from sagemaker import image_uris

region = boto3.Session().region_name    
container = image_uris.retrieve('randomcutforest', region)

rcf_estimator = Estimator(container,
                role=role,
                instance_count=1,
                instance_type='ml.m5.large',
                output_path='s3://{}/{}/output'.format(bucket, prefix))

rcf_estimator.set_hyperparameters(feature_dim=1)

In [95]:
rcf_estimator.fit(rcf_data)

2021-05-02 22:26:42 Starting - Starting the training job...
2021-05-02 22:27:08 Starting - Launching requested ML instancesProfilerReport-1619994402: InProgress
......
2021-05-02 22:28:14 Starting - Preparing the instances for training......
2021-05-02 22:29:28 Downloading - Downloading input data...
2021-05-02 22:29:55 Training - Downloading the training image..Docker entrypoint called with argument(s): train
Running default environment configuration script
[05/02/2021 22:30:15 INFO 140128021849920] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-conf.json: {'num_samples_per_tree': 256, 'num_trees': 100, 'force_dense': 'true', 'eval_metrics': ['accuracy', 'precision_recall_fscore'], 'epochs': 1, 'mini_batch_size': 1000, '_log_level': 'info', '_kvstore': 'dist_async', '_num_kv_servers': 'auto', '_num_gpus': 'auto', '_tuning_objective_metric': '', '_ftp_port': 8999}
[05/02/2021 22:30:15 INFO 140128021849920] Merging with provided co


2021-05-02 22:30:30 Uploading - Uploading generated training model
2021-05-02 22:30:30 Completed - Training job completed
Training seconds: 69
Billable seconds: 69


In [96]:
from time import strftime, gmtime
timestamp = strftime('%d-%H-%M-%S', gmtime())

endpoint_name = 'rcf-demo'+'-'+timestamp

rcf_predictor = rcf_estimator.deploy(endpoint_name=endpoint_name, 
                        initial_instance_count=1, 
                        instance_type='ml.t2.medium')

-----------------!

After a few minutes, the model is deployed. We convert the input time series to
a Python list, and we send it to the endpoint for prediction. We use CSV and JSON,
respectively, for serialization and deserialization:

In [105]:
rcf_predictor.serializer = sagemaker.serializers.CSVSerializer()
rcf_predictor.deserializer = sagemaker.deserializers.JSONDeserializer()

values = df['value'].astype('str').tolist()
response = rcf_predictor.predict(values)

#print("\nThe response contains the anomaly score for each value in the time series. It looks like this:\n")
#print(response)

We then convert this response to a Python list, and we then compute its mean and
its standard deviation:

In [106]:
from statistics import mean,stdev

scores = []
for s in response['scores']:
    scores.append(s['score'])
    
score_mean = mean(scores)
score_std = stdev(scores)

In [107]:
#df[2000:2500].plot(figsize=(40,10))

In [108]:
#plt.figure(figsize=(40,10))
#plt.plot(scores[2000:2500])
#plt.autoscale(tight=True)
#plt.axhline(y=score_mean+3*score_std, color='red')
#plt.show()

### deleting endpoint

In [109]:
rcf_predictor.delete_endpoint()